In [ ]:
# inflating training set with data augmentation

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


datagen = ImageDataGenerator(
        rotation_range = 10,
        shear_range = 0.1,
        zoom_range = 0.125,
        horizontal_flip = True,
        brightness_range = (0.75, 2.0))

for f in os.listdir('../input/exmlfinalstrain/train/terraria/'):
    img = load_img(f"../input/exmlfinalstrain/train/terraria/{f}")  
    x = img_to_array(img)
    x = x.reshape((1, ) + x.shape)  
    i = 0

    # generate 1 augmented image for each original image
    for batch in datagen.flow(x, batch_size = 1, 
                      save_to_dir ='terraria',  
                      save_prefix ='terraria', save_format ='jpg'): 
        i += 1
        if i > 1: 
            break


In [ ]:
# turning generated data to zip for downloading and adding to train data
import shutil
shutil.make_archive("terraria", 'zip', "terraria")

In [1]:
from fastai.vision.all import *
from fastai.vision.models.all import *
import numpy as np

np.random.seed(1)
path = Path(r'../input/exmlfinalstraining/train')
data = ImageDataLoaders.from_folder(path, valid_pct = 0.2, bs = 64, batch_tfms=Normalize.from_stats(*imagenet_stats), item_tfms=Resize(224,224))


In [3]:
learn = cnn_learner(data,vgg16_bn, metrics=accuracy, model_dir= '/kaggle/working/models')

In [4]:
learn.fine_tune(4)

In [5]:
learn.unfreeze()

In [6]:
learn.lr_find()

In [7]:
learn.fit_one_cycle(4, lr_max = slice(1e-04, 2.511886486900039e-05))

In [ ]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread("../input/exml-finals/Test_Data/Test_Data/ANNvIyctVbXjzemHBthu_73035950.jpg")

plt.imshow(img)

pred_class, pred_idx, outputs = learn.predict(img)
print(pred_class)

In [14]:
import cv2
import numpy as np
import os

res = {}
tally = 0

#Predicting on the whole test data for submission
for image in os.listdir("../input/exml-finals/Test_Data/Test_Data"):
    print(image, tally)
    img = cv2.imread(f"../input/exml-finals/Test_Data/Test_Data/{image}")
    img = cv2.resize(img, (224,224))
    pred, idx, outputs = learn.predict(img)
    print(image)
    res[image] = int(idx)+1
    tally += 1
    
print(res)


# predicting on one image
# image = cv2.imread("../input/exml-finals/Test_Data/Test_Data/ACVeSZuTSlAyjHBgoSZq_557828206.jpg")
# plt.imshow(image)
# image = cv2.resize(image, (224,224))
# # image = np.expand_dims(image, axis = 0)
# pred, idx, outputs = learn.predict(image)

# print(pred)


In [15]:
#generating submission.csv
import pandas as pd
df = pd.DataFrame(res.items(), columns=['id', 'game_id'])
df.to_csv("submission4forfinals.csv", index=False)